## Importamos el excel

In [1]:
import pandas as pd

# Ruta al archivo Excel
file_path = '/home/jovyan/work/transcripciones.xlsx'

# Cargamos el archivo Excel
df = pd.read_excel(file_path)

# Verificar que las columnas existen
assert 'Texto Original' in df.columns, "La columna 'Texto Original' no existe en el archivo Excel"
assert 'Texto Corregido' in df.columns, "La columna 'Texto Corregido' no existe en el archivo Excel"

df.head()


,Texto Original,Texto Corregido
0,[Música] esto es importante para cada adulto o...,Esto es importante para cada adulto o cada per...
1,ya hice un top al respecto hace muchos años y ...,Ya hice un top al respecto hace muchos años y ...
2,Qué pasa chicos eh estoy aquí un poco a oscura...,¿Qué pasa chicos? Estoy aquí un poco a oscuras...
3,mi gente de Perú qué tal estáis ayer nos queda...,"Mi gente de Perú, ¿qué tal estáis? Ayer nos qu..."
4,Estamos en uno de los trenes\nmás costosos de ...,Estamos en uno de los trenes más costosos de t...


## Dividimos el conjunto de datos en conjuntos de entrenamiento y validación

In [2]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.head()


,Texto Original,Texto Corregido
0,[Música] esto es importante para cada adulto o...,Esto es importante para cada adulto o cada per...
7,[Música] [Música] [Música] más [Música] voy a ...,"Hola, voy a explicarte todo lo que te extraño...."
2,Qué pasa chicos eh estoy aquí un poco a oscura...,¿Qué pasa chicos? Estoy aquí un poco a oscuras...
4,Estamos en uno de los trenes\nmás costosos de ...,Estamos en uno de los trenes más costosos de t...
3,mi gente de Perú qué tal estáis ayer nos queda...,"Mi gente de Perú, ¿qué tal estáis? Ayer nos qu..."


## Usaremos el modelos T5. Tokenizamos antes de introducir el texto en el modelo con T5Tokenizer 
### Se puede usar 't5-base' o 't5-large' según lo preciso (más tiempo de entrenamiento) que queramos entrenar el modelo 

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carga el tokenizador y el modelo
model_name = 't5-small'  
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Función para tokenizar y preparar los datos
def preprocess_data(data, tokenizer, max_len=512):
    inputs = [f"corrija: {text}" for text in data['Texto Original']]
    targets = [text for text in data['Texto Corregido']]
    
    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_len)
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_len)
    
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

train_encodings = preprocess_data(train_df, tokenizer)
val_encodings = preprocess_data(val_df, tokenizer)


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Creación del Dataset de entrenamiento y validacion

In [4]:
import torch
from torch.utils.data import Dataset

class SubtitlesDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

# Crea de entrenamiento y validacion
train_dataset = SubtitlesDataset(train_encodings)
val_dataset = SubtitlesDataset(val_encodings)


In [5]:
from transformers import Trainer, TrainingArguments

# Configurar los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida
    num_train_epochs=3,              # Epocas
    per_device_train_batch_size=8,   # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=8,    # Tamaño del lote de evaluación
    warmup_steps=500,                # Ppasos de calentamiento
    weight_decay=0.01,               # Desintegracion del peso
    logging_dir='./logs',            # Directorio de logs por si acaso
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Creamos el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Entrenamiento del modelo

In [6]:
# Iniciamos el entrenamiento
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,12.459884
2,No log,12.456382
3,No log,12.449466


TrainOutput(global_step=3, training_loss=10.818243662516275, metrics={'train_runtime': 21.8717, 'train_samples_per_second': 0.823, 'train_steps_per_second': 0.137, 'total_flos': 2436152426496.0, 'train_loss': 10.818243662516275, 'epoch': 3.0})

# Evaluación del modelo



In [7]:
eval_results = trainer.evaluate()
print(f"Resultados de la evaluación: {eval_results}")

Resultados de la evaluación: {'eval_loss': 12.44946575164795, 'eval_runtime': 0.4043, 'eval_samples_per_second': 4.947, 'eval_steps_per_second': 2.473, 'epoch': 3.0}


In [ ]:
# Almacenamos el modelo entrenado
# model.save_pretrained('/home/jovyan/work/modelo_corregido')
# tokenizer.save_pretrained('/home/jovyan/work/modelo_corregido')
